In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
# Define the CNN + LBP model architecture
def CNN_LBP_Model(input_shape, num_classes):
    model = tf.keras.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Flatten the output
    model.add(layers.Flatten())

    # LBP layer
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))

    # Fully connected layers
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 10

# Create an instance of the CNN + LBP model
model = CNN_LBP_Model(input_shape, num_classes)

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                   

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load your dataset using the ImageDataGenerator
train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Tomato_Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    '//content/drive/MyDrive/Colab Notebooks/Leaf Disease/Tomato_test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 8950 images belonging to 10 classes.
Found 2295 images belonging to 10 classes.


In [8]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=40,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/40
280/280 [==============================] - 3893s 14s/step - loss: 2.1852 - accuracy: 0.1641 - val_loss: 1.9609 - val_accuracy: 0.2702
Epoch 2/40
280/280 [==============================] - 178s 635ms/step - loss: 1.8556 - accuracy: 0.2893 - val_loss: 1.6361 - val_accuracy: 0.4144
Epoch 3/40
280/280 [==============================] - 174s 623ms/step - loss: 1.7061 - accuracy: 0.3684 - val_loss: 1.5168 - val_accuracy: 0.4614
Epoch 4/40
280/280 [==============================] - 175s 626ms/step - loss: 1.5642 - accuracy: 0.4198 - val_loss: 1.3714 - val_accuracy: 0.5054
Epoch 5/40
280/280 [==============================] - 176s 627ms/step - loss: 1.4498 - accuracy: 0.4761 - val_loss: 1.2532 - val_accuracy: 0.5599
Epoch 6/40
280/280 [==============================] - 178s 635ms/step - loss: 1.3134 - accuracy: 0.5225 - val_loss: 1.2122 - val_accuracy: 0.5477
Epoch 7/40
280/280 [==============================] - 176s 630ms/step - loss: 1.2039 - accuracy: 0.5636 - val_loss: 1.0074 - 

In [10]:
# Save the trained model
model.save('/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Models/CNN+LBP_1.h5')